In [1]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.models import Button, CustomJS, Div, TextInput
from bokeh.plotting import figure, output_file, save, show
from bokeh.events import Tap, ValueSubmit
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import column, row
from bokeh.events import ButtonClick, Tap, Press
import random
import os
import json
import numpy as np
import pandas as pd

In [2]:
f = open('data/name_to_link.json')
name_to_link = json.load(f)

f = open('data/clip_to_question.json')
clip_to_question = json.load(f)

f = open('data/video_to_scenario.json')
video_to_scenario = json.load(f)

pca_df = pd.read_csv("data/clip_q_to_tsne_q.csv")#"data/clip_q_to_pca_q.csv")
clip_q_to_pca = {row[0]: [row[1], row[2]] for index, row in pca_df.iterrows()}

In [3]:
AMOUNT = 1000
CIRCLE_SIZE = 30
IMAGE_SIZE = 26


def hover_page(scenario, q, image_src):
    hover_text = f"""<div id='main' style="display: flex; align-items: flex-start; border: 1px solid #ccc; padding: 10px; margin: 10px;">
    <div id='image' style="display: inline-block; vertical-align: top;">
        <img src="{image_src}" height="150" width="200" border="2" alt="Image"></img>
    </div>
    <div id='text' style="display: inline-block; width: 200px; word-wrap: break-word; vertical-align: top; margin-left: 10px; style="font-size: 10px;">
        <span style="font-weight: bold;">Question:</span><span>{q}</span><br>
        {scenario}
    </div>
</div>
"""
    return hover_text

TO_IGNORE= ['168b7587-f0e6-47b2-bb9b-831840f7eb8b_32351']
def check(clip_q):
    clip_f = clip_q[:clip_q.rfind("-")]
    if clip_f in TO_IGNORE:
        return False
    return True

def get_list(scenarios):
    if len(scenarios) == 0:
        return ""
    
    list_string = ""
    for scen in scenarios:
        list_string += f"<li>{scen}</li>\n"
    
    div = f"""<div style="margin-top: 10px;">
                 <span style="font-weight: bold;">Scenarios:</span>
                 <ul id="scenarios" style="margin-top: 5px; padding-left: 20px;">
                     {list_string}
                 </ul>
             </div>"""
    return div
    

clip_names = [clip_q for clip_q in clip_to_question if check(clip_q)][:AMOUNT]
x = [clip_q_to_pca[clip_q][0]/2 for clip_q in clip_names]
y = [clip_q_to_pca[clip_q][1]/2 for clip_q in clip_names]
scenarios = [get_list(video_to_scenario[f"{clip_q[:clip_q.rfind('_')]}"]) for clip_q in clip_names]
frame_locations = [f"transparent_images/{clip_q[:clip_q.rfind('-')]}.png" for clip_q in clip_names]
original_frame_locations = [f"images/{clip_q[:clip_q.rfind('-')]}.jpg" for clip_q in clip_names]
clip_url = [name_to_link[f"{clip_q[:clip_q.rfind('-')]}.mp4"] for clip_q in clip_names]
clip_q = []
clip_a = []
clip_wa = []
clip_wb = []
clip_wc = []
clip_wd = []

for clip_n in clip_names:
    qa_string = clip_to_question[clip_n]
    qaw = qa_string.split("\n")
    q = qaw[0][qaw[0].find(":") + 2:]
    a = qaw[1][qaw[1].find(":") + 2:]
    wa = qaw[2][qaw[2].find(":") + 2:]
    wb = qaw[3][qaw[3].find(":") + 2:]
    wc = qaw[4][qaw[4].find(":") + 2:]
    wd = qaw[5][qaw[5].find(":") + 2:]
    
    clip_q.append(q)
    clip_a.append(a)
    clip_wa.append(wa)
    clip_wb.append(wb)
    clip_wc.append(wc)
    clip_wd.append(wd)
    
hover_def = [hover_page(scenarios[i], clip_q[i], original_frame_locations[i]) for i in range(len(clip_names))]
hover_current = [hover_page(scenarios[i], clip_q[i], original_frame_locations[i]) for i in range(len(clip_names))]

In [7]:
data_1 = {'x':x,
        'y':y,
        'clip_names':clip_names,
        'url':frame_locations,
        'original_url':original_frame_locations,
        'clip_url':clip_url,
        'scenarios':scenarios,
        'q':clip_q,
        'a':clip_a,
        'wa':clip_wa,
        'wb':clip_wb,
        'wc':clip_wc,
        'wd':clip_wd,
        'alpha':[1]*AMOUNT,
        'hover_def':hover_def,
        'hover_current':hover_current
        }

data_2 = {'x':x,
        'y':y,
        'clip_names':clip_names,
        'url':frame_locations,
        'original_url':original_frame_locations,
        'clip_url':clip_url,
        'scenarios':scenarios,
        'q':clip_q,
        'a':clip_a,
        'wa':clip_wa,
        'wb':clip_wb,
        'wc':clip_wc,
        'wd':clip_wd,
        'alpha':[1]*AMOUNT,
        'hover_def':hover_def,
        'hover_current':hover_current
        }


source_1 = ColumnDataSource(data_1)
source_2 = ColumnDataSource(data_2)

f = open("html_txts/hover.txt", "r")
HOWER = f.read()

f = open("html_txts/js_code.txt", "r")
JS_CODE = f.read()

f = open("html_txts/search_code.txt", "r")
SEARCH_CODE = f.read()

p = figure(sizing_mode="stretch_both", 
           width=2500, 
           height=700, 
           tooltips=HOWER, 
           tools="pan,wheel_zoom,reset,hover,tap", active_scroll = "wheel_zoom")

p.toolbar_location = None

p.outline_line_width = 3
p.outline_line_alpha = 0.5
p.outline_line_color = "navy"

p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False

p.y_range.start = np.mean(y) - 1 * np.std(y)
p.y_range.end = np.mean(y) + 1 * np.std(y)
p.x_range.start = np.mean(x) - 1 * np.std(x)
p.x_range.end = np.mean(x) + 1 * np.std(x)


circle = p.circle(x = 'x', y='y', fill_color="black", color="black", size=CIRCLE_SIZE, alpha='alpha', source = source_1,)
images = p.image_url(url = 'url', x = 'x', y='y',
                     anchor="center", w=IMAGE_SIZE, w_units="screen", 
                     h=IMAGE_SIZE, h_units="screen", dilate=True, alpha='alpha',
                     source = source_2, )

images.selection_glyph = images.glyph
images.nonselection_glyph = images.glyph
images.muted_glyph = images.glyph
circle.selection_glyph = circle.glyph
circle.nonselection_glyph = circle.glyph

callback = CustomJS(args=dict(source_1=source_1, source_2 = source_2), code=JS_CODE)
search_callback = CustomJS(args=dict(source_1=source_1, source_2 = source_2), code=SEARCH_CODE)

#p.add_tools(TapTool(behavior='inspect', callback=callback))
source_1.selected.js_on_change('indices', callback)
#p.js_on_event(Tap, callback)
p.hover.renderers = [circle]

text_input = TextInput(placeholder="Search by keyword in answer options", width=1500, height = 50, css_classes = ["text-input-1223"])
text_input.sizing_mode = "stretch_width"
text_input.js_on_event(ValueSubmit, search_callback)

layout = column(p, text_input)
layout.sizing_mode = "stretch_both"

# save the results to a file
output_file(filename="plot.html", title="Static HTML file")
show(layout)

In [58]:
from bokeh.models import Circle
from bokeh.plotting import figure, show

plot = figure(width=400, height=400, tools="tap", title="Select a circle")
renderer = plot.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=50, fill_alpha=1, fill_color="blue", line_color=None)

selected_circle = renderer
nonselected_circle = renderer

renderer.selection_glyph = renderer.glyph
renderer.nonselection_glyph = renderer.glyph

show(plot)

In [6]:
data = {'x':x,
        'y':y,
        'clip_names':clip_names,
        'url':frame_locations,
        'original_url':original_frame_locations,
        'clip_url':clip_url,
        'scenarios':scenarios,
        'q':clip_q,
        'a':clip_a,
        'wa':clip_wa,
        'wb':clip_wb,
        'wc':clip_wc,
        'wd':clip_wd,
        'alpha':[1]*AMOUNT,
        'hover_def':hover_def,
        'hover_current':hover_current
        }

f = open("html_txts/js_code_check.txt", "r")
JS_CODE = f.read()

source = ColumnDataSource(data)


p = figure(sizing_mode="stretch_both", 
           width=2500, 
           height=700,
           tools="pan,wheel_zoom,reset,hover")

images = p.image_url(url = 'url', x = 'x', y='y',
                     anchor="center", w=IMAGE_SIZE, w_units="screen", 
                     h=IMAGE_SIZE, h_units="screen", dilate=True, alpha='alpha',
                     source = source, )

#images.selection_glyph = images.glyph
#images.nonselection_glyph = images.glyph

callback = CustomJS(args=dict(source=source), code=JS_CODE)
#source.selected.js_on_change('indices', callback)
p.add_tools(TapTool(behavior='select', renderers = [images], callback=callback))

output_file(filename="click_check.html", title="Static HTML file")
show(p)

In [35]:
from __future__ import annotations

import numpy as np

from bokeh import events
from bokeh.io import curdoc, show
from bokeh.layouts import column, row
from bokeh.models import Button, CustomJS, Div, TextInput
from bokeh.plotting import figure


def display_event(div: Div, attributes: list[str] = []) -> CustomJS:
    """
    Function to build a suitable CustomJS to display the current event
    in the div model.
    """
    style = 'float: left; clear: left; font-size: 13px'
    return CustomJS(args=dict(div=div), code=f"""
        const attrs = {attributes};
        const args = [];
        for (let i = 0; i < attrs.length; i++) {{
            const val = JSON.stringify(cb_obj[attrs[i]], function(key, val) {{
                return val.toFixed ? Number(val.toFixed(2)) : val;
            }})
            args.push(attrs[i] + '=' + val)
        }}
        const line = "<span style={style!r}><b>" + cb_obj.event_name + "</b>(" + args.join(", ") + ")</span>\\n";
        const text = div.text.concat(line);
        const lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");
    """)

# Follows the color_scatter gallery example

N = 4000
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors = np.array([(r, g, 150) for r, g in zip(50+2*x, 30+2*y)], dtype="uint8")

p = figure(tools="pan,wheel_zoom,zoom_in,zoom_out,reset,tap,lasso_select,box_select,box_zoom,undo,redo")

p.scatter(x, y, radius=radii,
          fill_color=colors, fill_alpha=0.6,
          line_color=None)

# Add a div to display events and a button to trigger button click events

div = Div(width=1000)
button = Button(label="Button", button_type="success", width=300)
text_input = TextInput(placeholder="Input a value and press Enter ...", width=300)
layout = column(button, text_input, row(p, div))

# Register event callbacks

# Button events
button.js_on_event(events.ButtonClick, display_event(div))

# TextInput events
text_input.js_on_event(events.ValueSubmit, display_event(div, ["value"]))

# LOD events
p.js_on_event(events.LODStart, display_event(div))
p.js_on_event(events.LODEnd, display_event(div))

# Point events
point_attributes = ['x','y','sx','sy']
p.js_on_event(events.Tap,       display_event(div, attributes=point_attributes))
p.js_on_event(events.DoubleTap, display_event(div, attributes=point_attributes))
p.js_on_event(events.Press,     display_event(div, attributes=point_attributes))
p.js_on_event(events.PressUp,   display_event(div, attributes=point_attributes))

# Mouse wheel event
p.js_on_event(events.MouseWheel, display_event(div,attributes=point_attributes+['delta']))

# Mouse move, enter and leave
# p.js_on_event(events.MouseMove,  display_event(div, attributes=point_attributes))
p.js_on_event(events.MouseEnter, display_event(div, attributes=point_attributes))
p.js_on_event(events.MouseLeave, display_event(div, attributes=point_attributes))

# Pan events
pan_attributes = point_attributes + ['delta_x', 'delta_y']
p.js_on_event(events.Pan,      display_event(div, attributes=pan_attributes))
p.js_on_event(events.PanStart, display_event(div, attributes=point_attributes))
p.js_on_event(events.PanEnd,   display_event(div, attributes=point_attributes))

# Pinch events
pinch_attributes = point_attributes + ['scale']
p.js_on_event(events.Pinch,      display_event(div, attributes=pinch_attributes))
p.js_on_event(events.PinchStart, display_event(div, attributes=point_attributes))
p.js_on_event(events.PinchEnd,   display_event(div, attributes=point_attributes))

# Ranges Update events
p.js_on_event(events.RangesUpdate, display_event(div, attributes=['x0','x1','y0','y1']))

# Selection events
p.js_on_event(events.SelectionGeometry, display_event(div, attributes=['geometry', 'final']))

curdoc().on_event(events.DocumentReady, display_event(div))

show(layout)

In [113]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.models import Button, CustomJS, Div, TextInput
from bokeh.plotting import figure, output_file, save, show
from bokeh.events import Tap, ValueSubmit
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import column, row
from bokeh.events import ButtonClick
import random
import os
import json
import numpy as np

TO_IGNORE= ['168b7587-f0e6-47b2-bb9b-831840f7eb8b_32351.mp4']
def check(clip_f):
    if clip_f[:clip_f.rfind(".")] in TO_IGNORE:
        return False
    return True

f = open('data_jsons/name_to_link.json')
name_to_link = json.load(f)

f = open('data_jsons/clip_to_question.json')
clip_to_question = json.load(f)

AMOUNT = 500
CIRCLE_SIZE = 40
IMAGE_SIZE = 36


x = random.sample(list(range(600)), AMOUNT)
y = np.array(random.sample(list(range(600)), AMOUNT)) / 2
clip_names = [clip_f[:clip_f.rfind(".")] for clip_f in os.listdir("transparent_images") if check(clip_f)][:AMOUNT]
frame_locations = [f"transparent_images/{clip_n}.png" for clip_n in clip_names]
original_frame_locations = [f"images/{clip_n}.jpg" for clip_n in clip_names]
clip_url = [name_to_link[f"{n}.mp4"] for n in clip_names]
clip_q = []
clip_a = []
clip_wa = []
clip_wb = []
clip_wc = []
clip_wd = []

for clip_n in clip_names:
    qa_string = clip_to_question[clip_n]
    qaw = qa_string.split("\n")
    q = qaw[0][qaw[0].find(":") + 2:]
    a = qaw[1][qaw[1].find(":") + 2:]
    wa = qaw[2][qaw[2].find(":") + 2:]
    wb = qaw[3][qaw[3].find(":") + 2:]
    wc = qaw[4][qaw[4].find(":") + 2:]
    wd = qaw[5][qaw[5].find(":") + 2:]
    
    clip_q.append(q)
    clip_a.append(a)
    clip_wa.append(wa)
    clip_wb.append(wb)
    clip_wc.append(wc)
    clip_wd.append(wd)
    
data = {'x':x,
        'y':y,
        'clip_names':clip_names,
        'url':frame_locations,
        'original_url':original_frame_locations,
        'clip_url':clip_url,
        'q':clip_q,
        'a':clip_a,
        'wa':clip_wa,
        'wb':clip_wb,
        'wc':clip_wc,
        'wd':clip_wd,
        'alpha':[1]*AMOUNT,
        }


source = ColumnDataSource(data)

f = open("html_txts/hover.txt", "r")
HOWER = f.read()

f = open("html_txts/js_code.txt", "r")
JS_CODE = f.read()

f = open("html_txts/search_code.txt", "r")
SEARCH_CODE = f.read()

p = figure(sizing_mode="stretch_both", 
           width=2500, 
           height=700, 
           tooltips=HOWER, 
           tools="pan,wheel_zoom,reset,tap,hover",)
p.outline_line_width = 3
p.outline_line_alpha = 0.5
p.outline_line_color = "navy"

p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False

images = p.image_url(url = 'url', x = 'x', y='y',
                     anchor="center", w=IMAGE_SIZE, w_units="screen", 
                     h=IMAGE_SIZE, h_units="screen", dilate=True, alpha='alpha',
                     source = source, )
circle = p.circle(x = 'x', y='y', fill_color="red", size=CIRCLE_SIZE, alpha='alpha', source = source,)

callback = CustomJS(args=dict(source=source), code=JS_CODE)
search_callback = CustomJS(args=dict(source=source, renderer=images), code=SEARCH_CODE)

source.selected.js_on_change('indices', callback)
p.hover.renderers = [circle]

text_input = TextInput(placeholder="Input a value and press Enter ...", width=1500, height = 50, css_classes = ["text-input-1223"])
text_input.sizing_mode = "stretch_width"
text_input.js_on_event(ValueSubmit, search_callback)

layout = column(p, text_input)
layout.sizing_mode = "stretch_both"

# save the results to a file
output_file(filename="custom_filename.html", title="Static HTML file")
show(layout)

FileNotFoundError: [Errno 2] No such file or directory: 'data_jsons/name_to_link.json'

In [149]:
f = open('name_to_link.json')
name_to_link = json.load(f)

In [150]:
x = random.sample(list(range(200)), 50)
y = random.sample(list(range(100)), 50)
clip_names = [clip_f[:clip_f.rfind(".")] for clip_f in os.listdir("transparent_images")][:50]
frame_locations = [f"transparent_images/{clip_n}.png" for clip_n in clip_names]
clip_urls = [name_to_link[f"{n}.mp4"] for n in clip_names]

In [159]:
source = ColumnDataSource(data=dict(
    x=x,
    y=y,
    clip_names=clip_names,
    url=frame_locations,
    clip_urls=clip_urls
))

In [160]:
f = open("hover.txt", "r")
HOWER = f.read()

In [161]:
callback = CustomJS(args=dict(source=source), code="""
window.alert(source.selected.indices);
var myIndex = source.selected.indices;
var myUrl = source.data.clip_urls[myIndex];
window.alert(myUrl);
window.location.href = "file:///C:/Users/artma/Desktop/School/website/clip.html?param1=" + encodeURIComponent(myUrl);;
""")

In [162]:
p = figure(sizing_mode="stretch_width", max_width=1100, height=600, tooltips=HOWER, tools="pan,wheel_zoom,reset,tap,hover",)

# add a circle renderer
circle = p.circle(source = source, fill_color="red", size=70, )
images = p.image_url(source = source, anchor="center", 
            w=67, w_units="screen", h=67, h_units="screen", dilate=True)
#taptool = p.select(type=TapTool)
#taptool.callback = callback
source.selected.js_on_change('indices', callback)
p.hover.renderers = [circle]

# save the results to a file
output_file(filename="custom_filename.html", title="Static HTML file")
show(p)

In [10]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.plotting import figure, output_file, save, show
from bokeh.events import Tap
from bokeh.models.callbacks import CustomJS

#Value: 'string_of_characters_like_these%3A%24%23%40%3D%3F%25%5EQ%5E%24'


source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    desc=['A', 'b', 'C', 'd', 'E'],
    url=[
        "transparent_images/023bf95e-28de-43b4-a43f-720edba667a5_51928.png",
        "transparent_images/023bf95e-28de-43b4-a43f-720edba667a5_51928.png",
        "transparent_images/023bf95e-28de-43b4-a43f-720edba667a5_51928.png",
        "transparent_images/023bf95e-28de-43b4-a43f-720edba667a5_51928.png",
        "transparent_images/023bf95e-28de-43b4-a43f-720edba667a5_51928.png"
    ],
))

f = open("hover.txt", "r")
HOWER = f.read()

callback = CustomJS(args=dict(source=source), code="""
var myUrl = "https://drive.google.com/uc?export=view&id=1PqERbEceLHiO1bhBxqWxwuCUWp7XxAKt";
window.location.href = "file:///C:/Users/artma/Desktop/School/website/clip.html?param1=" + encodeURIComponent(myUrl);;
""")


p = figure(sizing_mode="stretch_width", max_width=500, height=500, tooltips=HOWER, tools="pan,wheel_zoom,reset,tap,hover",)

# add a circle renderer
circle = p.circle(source = source, fill_color="red", size=70, )
images = p.image_url(source = source, anchor="center", 
            w=45, w_units="screen", h=45, h_units="screen", dilate=True)
taptool = p.select(type=TapTool)
taptool.callback = callback
p.hover.renderers = [circle]

# save the results to a file
output_file(filename="custom_filename.html", title="Static HTML file")
show(p)